In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, Flatten
import pickle

In [2]:
IoT = pd.read_csv('../../data/IoT_GPS_Tracker.csv')
IoT = IoT.iloc[0:500]

IoT = IoT.dropna()

encoder=LabelEncoder()
IoT['type']=encoder.fit_transform(IoT['type'])


IoT.head()

,date,time,latitude,longitude,label,type
0,31-Mar-19,12:36:52,0.0,10.0,0,0
1,31-Mar-19,12:36:53,0.0,10.0,0,0
2,31-Mar-19,12:36:54,0.0,10.0,0,0
3,31-Mar-19,12:36:55,0.0,10.0,0,0
4,31-Mar-19,12:36:56,0.0,10.0,0,0


In [3]:
x = IoT.drop(['label', 'date', 'time', 'type'], axis=1)
y = IoT['label']

features=len(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [4]:
model2 = Sequential()
    
model2.add(GRU(units = 128, activation='tanh', return_sequences=True, input_shape = (features,1)))
model2.add(Dropout(0.2))
model2.add(GRU(units = 64, activation='tanh', return_sequences=True))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation='sigmoid')) 
model2.add(Flatten())
    
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
model2.save('../h5s/GRU.h5') 
# filename = '../h5s/GRU.h5'
# pickle.dump(model2, open(filename, 'wb'))

In [6]:
model2.fit(x_train, np.asarray(y_train).astype('float32').reshape((-1,1)), epochs = 5, batch_size=32, validation_data=(x_test, np.asarray(y_test).astype('float32').reshape((-1,1))))

Epoch 1/5
13/13 [==============================] - 4s 62ms/step - loss: 0.3657 - accuracy: 1.0000 - val_loss: 0.2613 - val_accuracy: 1.0000
Epoch 2/5
13/13 [==============================] - 0s 7ms/step - loss: 0.2211 - accuracy: 1.0000 - val_loss: 0.1708 - val_accuracy: 1.0000
Epoch 3/5
13/13 [==============================] - 0s 7ms/step - loss: 0.1319 - accuracy: 1.0000 - val_loss: 0.0864 - val_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 0s 7ms/step - loss: 0.0615 - accuracy: 1.0000 - val_loss: 0.0359 - val_accuracy: 1.0000
Epoch 5/5
13/13 [==============================] - 0s 7ms/step - loss: 0.0254 - accuracy: 1.0000 - val_loss: 0.0141 - val_accuracy: 1.0000


In [7]:
predictions = model2.predict(x_test)
y_pred = [round(x[0]) for x in predictions]
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [8]:
for i in range(3):
	print(x_test.values[i].tolist(), '=>', y_pred[i], '( expected ', y_train.values[i], ')')

[0.262991527, 10.46781331] => 0 ( expected  0 )
[0.033850164, 10.06290894] => 0 ( expected  0 )
[0.01190555, 10.24440769] => 0 ( expected  0 )


In [9]:
score, acc = model2.evaluate(x_test, y_test, batch_size=1)

print('Test score:', score)
print('Test accuracy:', acc)

100/100 [==============================] - 1s 2ms/step - loss: 0.0141 - accuracy: 1.0000
Test score: 0.014071844518184662
Test accuracy: 1.0
